In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score,RandomizedSearchCV,RepeatedKFold
from sklearn.linear_model import Ridge,Lasso, LassoCV,LinearRegression
from xgboost import XGBRegressor
from model_functions import label_encoding,training_model,nn_models
from sklearn.model_selection import train_test_split
dataset=pd.read_excel("dataset.xlsx")
a=pd.read_excel("sample2.xlsx")

[NbConvertApp] Converting notebook model_functions.ipynb to script
[NbConvertApp] Writing 2362 bytes to model_functions.py


In [2]:
label_encoding(dataset, a, 'Destination')
label_encoding(dataset, a, 'Origin')
label_encoding(dataset, a, 'To Area')
label_encoding(dataset, a, 'day_name')
label_encoding(dataset, a, 'flight_month')

label_encoding(dataset, a, 'season')
label_encoding(dataset, a, 'Netto Currency')
label_encoding(dataset, a, 'Flight Code')

label_encoding(dataset, a, 'Airline Company')
label_encoding(dataset, a, 'Flight Date')

label_encoding(dataset, a, 'dpt')
label_encoding(dataset, a, 'dpt1')

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9376 entries, 0 to 9375
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Destination      9376 non-null   int64  
 1   Origin           9376 non-null   int64  
 2   To Area          9376 non-null   int64  
 3   Flight Date      9376 non-null   int64  
 4   day_name         9376 non-null   int64  
 5   flight_month     9376 non-null   int64  
 6   season           9376 non-null   int64  
 7   year             9376 non-null   int64  
 8   Flight Code      9376 non-null   int64  
 9   Days             9376 non-null   int64  
 10  Airline Company  9376 non-null   int64  
 11  dpt              9376 non-null   int64  
 12  Block            9376 non-null   int64  
 13  Sold             9376 non-null   int64  
 14  Left             9376 non-null   int64  
 15  Occ.(%)          9376 non-null   int64  
 16  dpt1             9376 non-null   int64  
 17  Block1        

In [4]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Destination      11 non-null     int64  
 1   Origin           11 non-null     int64  
 2   To Area          11 non-null     int64  
 3   Flight Date      11 non-null     int64  
 4   day_name         11 non-null     int64  
 5   flight_month     11 non-null     int64  
 6   season           11 non-null     int64  
 7   year             11 non-null     int64  
 8   Flight Code      11 non-null     int64  
 9   Days             11 non-null     int64  
 10  Airline Company  11 non-null     int64  
 11  dpt              11 non-null     int64  
 12  Block            11 non-null     int64  
 13  Sold             11 non-null     int64  
 14  Left             11 non-null     int64  
 15  Occ.(%)          11 non-null     int64  
 16  dpt1             11 non-null     int64  
 17  Block1           1

In [5]:
dataset = dataset.drop(['day_convert'], axis=1)

In [6]:
dataset["Profit"]=dataset["Profit"].fillna(dataset["Profit"].median())

In [7]:
X = dataset.drop(['Occ.'], axis=1) # independent variables
y = dataset[['Occ.']]  # dependent variables
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.33, random_state=42)

In [8]:
X

,Destination,Origin,To Area,Flight Date,day_name,flight_month,season,year,Flight Code,Days,...,Occ.(%),dpt1,Block1,Sold1,Left1,Occ.(%)1,Netto,Netto Currency,Profit,prıce
0,0,0,0,0,0,0,0,2020,0,4,...,95,0,220,151,69,69,177.00,0,61.73,238.73
1,0,0,0,1,0,0,0,2020,0,4,...,0,0,220,220,0,100,174.00,0,229.11,174.00
2,0,1,0,0,0,0,0,2020,1,4,...,98,0,220,181,39,82,253.00,0,58.35,311.35
3,0,1,0,2,1,0,0,2020,1,5,...,0,0,220,217,3,99,236.00,0,229.11,236.00
4,0,1,0,3,2,1,1,2020,2,7,...,99,0,220,1,219,0,251.00,0,-50.69,200.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9371,0,5,0,496,6,9,3,2022,152,6,...,100,12,65,65,0,100,377.18,0,420.95,798.13
9372,0,5,0,496,6,9,3,2022,149,6,...,100,26,46,46,0,100,343.18,0,398.44,741.62
9373,0,5,0,497,2,9,3,2022,151,7,...,100,47,34,34,0,100,370.00,1,512.82,882.82
9374,0,5,0,497,2,9,3,2022,148,7,...,100,188,54,54,0,100,377.18,0,467.51,844.69


In [9]:
y

,Occ.
0,95.45
1,0.00
2,97.73
3,0.00
4,99.09
...,...
9371,100.00
9372,100.00
9373,100.00
9374,100.00


In [10]:
rf = RandomForestRegressor(n_estimators = 23, max_depth = 30,min_samples_split=5,random_state = 42)
DT_Regressor = tree.DecisionTreeRegressor()
ridge = Ridge(alpha=1.0)
lasso_cv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lr = LinearRegression()

In [11]:
xgb = XGBRegressor()

param_grid = {
    "learning_rate": [0.0001,0.001, 0.01, 0.1, 1] ,
    "max_depth": range(3,21,3),
    "gamma": [i/10.0 for i in range(0,5)],
    "colsample_bytree": [i/10.0 for i in range(3,10)],
    "reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100],
    "reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100]
}

xgb_cv = RandomizedSearchCV(xgb, param_grid, n_iter = 100, refit='recall',
         scoring='neg_mean_squared_error', cv = 5, verbose=2, random_state=42, n_jobs = -1) 
xgb_cv.fit(X_train, y_train)
xgb_reg = XGBRegressor(**xgb_cv.best_params_ )

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [12]:
models = { 'Random forest': rf,'Decision tree': DT_Regressor,'Ridge regression': ridge,
          'Lasso regression': lasso_cv,'Linear regression': lr,'XGBOOST': xgb_reg}

In [13]:
sample2 = a.drop(['Occ.'], axis=1) 
sample2

,Destination,Origin,To Area,Flight Date,day_name,flight_month,season,year,Flight Code,Days,...,Occ.(%),dpt1,Block1,Sold1,Left1,Occ.(%)1,Netto,Netto Currency,Profit,prıce
0,0,27,0,325,5,6,2,2021,148,3,...,65,28,220,212,8,96,149.00,1,-32.53,116.47
1,0,28,0,481,4,8,3,2022,127,2,...,71,170,189,189,0,100,246.00,0,-42.98,203.02
2,0,29,0,703,2,12,4,2022,152,7,...,100,69,58,58,0,100,475.82,0,562.33,1038.15
3,0,30,0,552,6,6,2,2022,31,6,...,100,23,14,14,0,100,660.36,0,647.46,1307.82
4,0,31,0,210,1,5,2,2021,9,5,...,100,26,163,163,0,100,356.36,0,318.20,674.56
5,0,32,0,561,3,4,3,2022,122,1,...,99,64,478,8,470,2,162.00,0,81.50,243.50
6,0,33,0,560,2,5,2,2022,128,7,...,100,71,474,0,474,0,150.00,0,153.87,303.87
7,0,34,0,490,2,5,2,2022,105,7,...,99,156,328,322,6,98,162.00,1,22.08,184.08
8,0,35,0,507,5,9,3,2022,148,3,...,96,188,110,82,28,75,200.00,0,75.78,275.78
9,0,36,0,532,2,9,3,2022,110,7,...,23,165,235,235,0,100,259.00,0,68.88,327.88


In [14]:
model_outputs = {}
sample_predicted={}

for i, (key, model) in enumerate(models.items()):
    model, score, mse, rmse, mae = training_model(X_train, X_test, y_train, y_test, model=model)
    model_outputs[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
    sample_predicted[key]=model.predict(sample2)

R^squared: 0.9999412137945068
MSE:  0.019776159642935023
RMSE:  0.14062773425940925
MAE: 0.042945902476942285
R^squared: 0.9999162064913039
MSE:  0.028188820678513818
RMSE:  0.1678952669925922
MAE: 0.0454474959612295
R^squared: 0.9999321011434565
MSE:  0.022841729880548008
RMSE:  0.15113480697889553
MAE: 0.07418374294832374


C:\Users\90541\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


R^squared: 0.9999310533141627
MSE:  0.023194228212768238
RMSE:  0.15229651411889977
MAE: 0.07777834358277923
R^squared: 0.9999320917574066
MSE:  0.022844887424396373
RMSE:  0.1511452527352294
MAE: 0.07423272453192054
R^squared: 0.9999322011204053
MSE:  0.022808096818468584
RMSE:  0.15102349757063827
MAE: 0.04928218888711665


In [15]:
model_outputs = pd.DataFrame(model_outputs)
model_outputs

,Random forest,Decision tree,Ridge regression,Lasso regression,Linear regression,XGBOOST
R-squared,0.999941,0.999916,0.999932,0.999931,0.999932,0.999932
MSE,0.019776,0.028189,0.022842,0.023194,0.022845,0.022808
RMSE,0.140628,0.167895,0.151135,0.152297,0.151145,0.151023
MAE,0.042946,0.045447,0.074184,0.077778,0.074233,0.049282


In [16]:
sample_predicted = pd.DataFrame(sample_predicted)

In [17]:
real_pred=pd.concat([a[['Occ.']],sample_predicted], axis=1)
real_pred

,Occ.,Random forest,Decision tree,Ridge regression,Lasso regression,Linear regression,XGBOOST
0,65.45,64.955170,65.15,64.986269,65.020637,64.984408,64.696053
1,70.90,70.867657,70.91,70.992115,71.015619,70.989913,71.365166
2,100.00,100.000000,100.00,99.985898,99.986159,99.983943,99.996536
3,100.00,100.000000,100.00,99.965786,99.986159,99.963075,99.997612
4,100.00,100.000000,100.00,99.940899,99.985082,99.938278,99.996582
5,99.39,99.372941,99.39,98.902796,98.987175,98.901536,99.317116
6,99.58,99.586739,99.58,99.910810,99.986159,99.909278,99.578796
7,99.39,99.370217,99.39,98.972141,98.987175,98.969371,99.107887
8,95.80,96.003873,96.30,95.984824,95.990222,95.982059,96.082130
9,23.40,22.879308,22.73,23.005094,23.064379,23.002448,23.156067


## mlp-lstm-rnn

In [18]:
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from tensorflow.python.keras.layers import Dense, LSTM, RepeatVector, Flatten,SimpleRNN

In [19]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
print('Train shape', X_train_scaled.shape)
print('Test shape', X_test_scaled.shape)

Train shape (6281, 25)
Test shape (3095, 25)


In [21]:
# input [samples, time steps, features] for LSTM
X_train_items = np.reshape(X_train_scaled, (X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_items = np.reshape(X_test_scaled, (X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

In [22]:
from sklearn.neural_network import MLPRegressor
mlp = MLPRegressor(hidden_layer_sizes=(500, 50), max_iter=2000, random_state=42)

training_model(X_train_scaled, X_test_scaled, y_train, y_test ,mlp)
sample_data = scaler.transform(sample2)
predictions = mlp.predict(sample_data)

In [23]:
modellstm = Sequential()
modellstm.add(LSTM(50, input_shape=(X_train_items.shape[1], 1)))
modellstm.add(Dense(1))
modellstm.compile(optimizer='adam', loss='mean_squared_error')

In [24]:
rnn = Sequential()
rnn.add(SimpleRNN(units=20, input_shape=(X_train_items.shape[1], X_train_items.shape[2])))
rnn.add(Dense(1))
rnn.compile(loss='mean_squared_error', optimizer='adam')

In [25]:
models = { 'MLP': mlp,'LSTM': modellstm,'RNN': rnn}

In [26]:
model_outputs_nn = {}
sample_predicted_nn={}

for i, (key, model) in enumerate(models.items()):
    if key == 'MLP':
        model, score, mse, rmse, mae=training_model(X_train_scaled, X_test_scaled, y_train, y_test ,model)
        model_outputs_nn[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
        sample_data = scaler.transform(sample2)
        sample_predicted_nn[key]=model.predict(sample_data)
    else:
        model, score, mse, rmse, mae =nn_models(X_train_scaled, X_test_scaled, y_train, y_test, model, epoch_num=200, batch_size=16)
        model_outputs_nn[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
        sample_predicted_nn[key]=model.predict(sample2)

R^squared: 0.9998429885379043
MSE:  0.05281993818327457
RMSE:  0.22982588666917958
MAE: 0.16153498938828023
Epoch 1/200
393/393 [==============================] - 19s 34ms/step - loss: 6411.5400
Epoch 2/200
393/393 [==============================] - 14s 36ms/step - loss: 3798.3123
Epoch 3/200
393/393 [==============================] - 13s 33ms/step - loss: 2247.0933
Epoch 4/200
393/393 [==============================] - 12s 32ms/step - loss: 1296.7391
Epoch 5/200
393/393 [==============================] - 12s 31ms/step - loss: 760.7768
Epoch 6/200
393/393 [==============================] - 11s 28ms/step - loss: 492.1812
Epoch 7/200
393/393 [==============================] - 11s 28ms/step - loss: 377.5273
Epoch 8/200
393/393 [==============================] - 11s 29ms/step - loss: 338.0033
Epoch 9/200
393/393 [==============================] - 12s 31ms/step - loss: 327.6563
Epoch 10/200
393/393 [==============================] - 11s 28ms/step - loss: 325.6842
Epoch 11/200
393/393 [=====

Epoch 95/200
393/393 [==============================] - 12s 32ms/step - loss: 320.4908
Epoch 96/200
393/393 [==============================] - 11s 28ms/step - loss: 312.4555
Epoch 97/200
393/393 [==============================] - 11s 29ms/step - loss: 305.1225
Epoch 98/200
393/393 [==============================] - 12s 31ms/step - loss: 301.4032
Epoch 99/200
393/393 [==============================] - 11s 28ms/step - loss: 296.5451
Epoch 100/200
393/393 [==============================] - 12s 32ms/step - loss: 291.3379
Epoch 101/200
393/393 [==============================] - 11s 28ms/step - loss: 283.5771
Epoch 102/200
393/393 [==============================] - 11s 29ms/step - loss: 274.7753
Epoch 103/200
393/393 [==============================] - 11s 28ms/step - loss: 266.2764
Epoch 104/200
393/393 [==============================] - 12s 31ms/step - loss: 259.7799
Epoch 105/200
393/393 [==============================] - 11s 29ms/step - loss: 257.1606
Epoch 106/200
393/393 [==============

393/393 [==============================] - 13s 33ms/step - loss: 10.5743
Epoch 190/200
393/393 [==============================] - 12s 32ms/step - loss: 10.4247
Epoch 191/200
393/393 [==============================] - 12s 30ms/step - loss: 5.5666
Epoch 192/200
393/393 [==============================] - 13s 33ms/step - loss: 5.0395
Epoch 193/200
393/393 [==============================] - 13s 32ms/step - loss: 5.6385
Epoch 194/200
393/393 [==============================] - 13s 33ms/step - loss: 5.3013
Epoch 195/200
393/393 [==============================] - 12s 31ms/step - loss: 4.3048
Epoch 196/200
393/393 [==============================] - 12s 31ms/step - loss: 2.7427
Epoch 197/200
393/393 [==============================] - 14s 35ms/step - loss: 2.0441
Epoch 198/200
393/393 [==============================] - 16s 40ms/step - loss: 6.0813
Epoch 199/200
393/393 [==============================] - 14s 35ms/step - loss: 2.0151
Epoch 200/200
97/97 [==============================] - 1s 5ms/step

393/393 [==============================] - 1s 2ms/step - loss: 129.5568
Epoch 83/200
393/393 [==============================] - 1s 2ms/step - loss: 99.0158
Epoch 84/200
393/393 [==============================] - 1s 2ms/step - loss: 72.7639
Epoch 85/200
393/393 [==============================] - 1s 2ms/step - loss: 62.0991
Epoch 86/200
393/393 [==============================] - 1s 2ms/step - loss: 54.9092
Epoch 87/200
393/393 [==============================] - 1s 3ms/step - loss: 47.9215
Epoch 88/200
393/393 [==============================] - 1s 2ms/step - loss: 44.2345
Epoch 89/200
393/393 [==============================] - 1s 2ms/step - loss: 41.0995
Epoch 90/200
393/393 [==============================] - 1s 2ms/step - loss: 38.2985
Epoch 91/200
393/393 [==============================] - 1s 2ms/step - loss: 37.0569
Epoch 92/200
393/393 [==============================] - 1s 2ms/step - loss: 38.0445
Epoch 93/200
393/393 [==============================] - 1s 2ms/step - loss: 32.9975
Epoc

393/393 [==============================] - 1s 2ms/step - loss: 1.8261
Epoch 180/200
393/393 [==============================] - 1s 2ms/step - loss: 1.5166
Epoch 181/200
393/393 [==============================] - 1s 2ms/step - loss: 1.6641
Epoch 182/200
393/393 [==============================] - 1s 3ms/step - loss: 1.5150
Epoch 183/200
393/393 [==============================] - 1s 2ms/step - loss: 1.7665
Epoch 184/200
393/393 [==============================] - 1s 3ms/step - loss: 1.9246
Epoch 185/200
393/393 [==============================] - 1s 3ms/step - loss: 1.5008
Epoch 186/200
393/393 [==============================] - 1s 3ms/step - loss: 1.4945
Epoch 187/200
393/393 [==============================] - 1s 3ms/step - loss: 1.4978
Epoch 188/200
393/393 [==============================] - 1s 3ms/step - loss: 1.2988
Epoch 189/200
393/393 [==============================] - 1s 2ms/step - loss: 1.5524
Epoch 190/200
393/393 [==============================] - 1s 3ms/step - loss: 1.2997
Epoch 

In [40]:
model_outputs_nn = pd.DataFrame(model_outputs_nn)
model_outputs_nn

,MLP,LSTM,RNN
R-squared,0.999843,0.993661,0.990878
MSE,0.052820,2.132615,3.068855
RMSE,0.229826,1.460348,1.751815
MAE,0.161535,0.954424,1.100604


In [45]:
sample_predicted_nn

{'MLP': array([ 65.08012646,  71.12291958,  99.96794486,  98.75865411,
         99.40327189, 100.15074386, 101.12020297,  99.44519998,
         95.60790053,  20.48921033,  51.79398514]),
 'LSTM': array([[ 4.317101 ],
        [ 3.5664444],
        [ 5.873078 ],
        [11.031271 ],
        [ 6.0092354],
        [ 5.258825 ],
        [ 5.3275504],
        [ 5.402327 ],
        [ 5.6651707],
        [ 5.888509 ],
        [ 3.260572 ]], dtype=float32),
 'RNN': array([[60.39218 ],
        [60.392174],
        [60.392174],
        [60.392174],
        [60.392174],
        [60.392174],
        [60.392174],
        [60.392097],
        [60.392174],
        [60.392174],
        [60.392174]], dtype=float32)}

In [46]:
sample_predicted_nn['LSTM'] = sample_predicted_nn['LSTM'].flatten()
sample_predicted_nn['RNN'] = sample_predicted_nn['RNN'].flatten()

In [50]:
sample_predicted_nn = pd.DataFrame(sample_predicted_nn)
real_pred_nn=pd.concat([a[['Occ.']],sample_predicted_nn], axis=1)
real_pred_nn

,Occ.,MLP,LSTM,RNN
0,65.45,65.080126,4.317101,60.392181
1,70.90,71.122920,3.566444,60.392174
2,100.00,99.967945,5.873078,60.392174
3,100.00,98.758654,11.031271,60.392174
4,100.00,99.403272,6.009235,60.392174
5,99.39,100.150744,5.258825,60.392174
6,99.58,101.120203,5.327550,60.392174
7,99.39,99.445200,5.402327,60.392097
8,95.80,95.607901,5.665171,60.392174
9,23.40,20.489210,5.888509,60.392174


## Dataset 
#x=zamansal girdiler ve Netto y=price

In [27]:
dataset1=dataset.loc[:,['Flight Date','day_name','flight_month','season','year','Days','dpt','dpt1','Netto','prıce']]
dataset1.head()

,Flight Date,day_name,flight_month,season,year,Days,dpt,dpt1,Netto,prıce
0,0,0,0,0,2020,4,0,0,177.0,238.73
1,1,0,0,0,2020,4,0,0,174.0,174.00
2,0,0,0,0,2020,4,1,0,253.0,311.35
3,2,1,0,0,2020,5,1,0,236.0,236.00
4,3,2,1,1,2020,7,0,0,251.0,200.31


In [28]:
X = dataset1.drop(['prıce'], axis=1) 
y = dataset1.iloc[:,-1:] 
X_train2, X_test2, y_train2, y_test2 = train_test_split(X,y.values.ravel(), test_size=0.33, random_state=42)

In [29]:
X

,Flight Date,day_name,flight_month,season,year,Days,dpt,dpt1,Netto
0,0,0,0,0,2020,4,0,0,177.00
1,1,0,0,0,2020,4,0,0,174.00
2,0,0,0,0,2020,4,1,0,253.00
3,2,1,0,0,2020,5,1,0,236.00
4,3,2,1,1,2020,7,0,0,251.00
...,...,...,...,...,...,...,...,...,...
9371,496,6,9,3,2022,6,24,12,377.18
9372,496,6,9,3,2022,6,190,26,343.18
9373,497,2,9,3,2022,7,117,47,370.00
9374,497,2,9,3,2022,7,75,188,377.18


In [30]:
X.columns

Index(['Flight Date', 'day_name', 'flight_month', 'season', 'year', 'Days',
       'dpt', 'dpt1', 'Netto'],
      dtype='object')

In [31]:
y

,prıce
0,238.73
1,174.00
2,311.35
3,236.00
4,200.31
...,...
9371,798.13
9372,741.62
9373,882.82
9374,844.69


In [32]:
sample3 = a.loc[:,['Flight Date', 'day_name', 'flight_month', 'season', 'year', 'Days',
       'dpt', 'dpt1', 'Netto']]
sample3

,Flight Date,day_name,flight_month,season,year,Days,dpt,dpt1,Netto
0,325,5,6,2,2021,3,26,28,149.00
1,481,4,8,3,2022,2,156,170,246.00
2,703,2,12,4,2022,7,24,69,475.82
3,552,6,6,2,2022,6,123,23,660.36
4,210,1,5,2,2021,5,24,26,356.36
5,561,3,4,3,2022,1,35,64,162.00
6,560,2,5,2,2022,7,88,71,150.00
7,490,2,5,2,2022,7,69,156,162.00
8,507,5,9,3,2022,3,190,188,200.00
9,532,2,9,3,2022,7,45,165,259.00


In [33]:
rf2 = RandomForestRegressor(n_estimators = 7, max_depth = 30,min_samples_split=5,random_state = 42)
DT_Regressor = tree.DecisionTreeRegressor()
ridge = Ridge(alpha=1.0)
lasso_cv = LassoCV(alphas = None, cv = 10, max_iter = 100000, normalize = True)
lr = LinearRegression()
models = { 'Random forest': rf2,'Decision tree': DT_Regressor,'Ridge regression': ridge,
          'Lasso regression': lasso_cv,'Linear regression': lr,'XGBOOST': xgb_reg}

In [34]:
model_outputs2 = {}
sample_predicted2={}

for i, (key, model) in enumerate(models.items()):
    model, score, mse, rmse, mae = training_model(X_train2, X_test2, y_train2, y_test2 , model=model)
    model_outputs2[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
    sample_predicted2[key]=model.predict(sample3)

R^squared: 0.8900708234010583
MSE:  10038.708768320093
RMSE:  100.19335690713278
MAE: 60.75765245361859
R^squared: 0.8205371711244561
MSE:  16388.506942003194
RMSE:  128.01760403164556
MAE: 77.0298061389337
R^squared: 0.7057997796519864
MSE:  26866.300858636107
RMSE:  163.90942882774044
MAE: 123.14051230589598
R^squared: 0.7057926958653206
MSE:  26866.947748511164
RMSE:  163.91140213088033
MAE: 123.1500624538953
R^squared: 0.7058026469516419
MSE:  26866.039017448144
RMSE:  163.9086300883762
MAE: 123.14230334234634


C:\Users\90541\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


R^squared: 0.8867571815018817
MSE:  10341.309834001664
RMSE:  101.69223094219963
MAE: 58.13741372835341


In [35]:
model_outputs2 = pd.DataFrame(model_outputs2)
model_outputs2

,Random forest,Decision tree,Ridge regression,Lasso regression,Linear regression,XGBOOST
R-squared,0.890071,0.820537,0.705800,0.705793,0.705803,0.886757
MSE,10038.708768,16388.506942,26866.300859,26866.947749,26866.039017,10341.309834
RMSE,100.193357,128.017604,163.909429,163.911402,163.908630,101.692231
MAE,60.757652,77.029806,123.140512,123.150062,123.142303,58.137414


In [36]:
sample_predicted2 = pd.DataFrame(sample_predicted2)

In [37]:
real_pred=pd.concat([a[['prıce']],sample_predicted2], axis=1)
real_pred

,prıce,Random forest,Decision tree,Ridge regression,Lasso regression,Linear regression,XGBOOST
0,116.47,152.975929,140.26,221.597274,222.303850,221.639390,211.741592
1,203.02,511.263476,688.55,543.138838,542.852175,543.372799,491.367340
2,1038.15,680.992714,920.26,722.886516,726.904661,722.149970,702.169739
3,1307.82,1248.962869,1115.19,1085.623522,1083.774199,1085.693297,1147.304077
4,674.56,390.040952,390.25,602.918853,600.081850,603.272653,379.074982
5,243.50,706.916548,739.19,447.894319,448.221469,448.129691,704.586731
6,303.87,685.033810,701.41,405.121645,404.188273,405.450326,687.676025
7,184.08,696.158000,689.08,451.363241,449.746205,451.874700,633.375061
8,275.78,614.844871,613.48,449.268383,449.793270,449.379824,540.868103
9,327.88,484.611960,369.36,530.596938,531.188766,530.685765,521.367310


In [59]:
X_train_scaled2 = scaler.fit_transform(X_train2)
X_test_scaled2 = scaler.transform(X_test2)

In [60]:
X_train_items2 = np.reshape(X_train_scaled2, (X_train_scaled2.shape[0], X_train_scaled2.shape[1], 1))
X_test_items2 = np.reshape(X_test_scaled2, (X_test_scaled2.shape[0], X_test_scaled2.shape[1], 1))

In [67]:
models = { 'MLP': mlp,'LSTM': modellstm,'RNN': rnn}

In [68]:
model_outputs_nn2 = {}
sample_predicted_nn2 ={}

for i, (key, model) in enumerate(models.items()):
    if key == 'MLP':
        model, score, mse, rmse, mae=training_model(X_train_scaled2, X_test_scaled2, y_train2, y_test2 ,model)
        model_outputs_nn2[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
        sample_data = scaler.transform(sample3)
        sample_predicted_nn2[key]=model.predict(sample_data)
    else:
        model, score, mse, rmse, mae =nn_models(X_train_scaled2, X_test_scaled2, y_train2, y_test2, model, epoch_num=200, batch_size=16)
        model_outputs_nn2[key] = {'R-squared': score, 'MSE': mse, 'RMSE': rmse, 'MAE': mae}
        sample_predicted_nn2[key]=model.predict(sample3)
     

R^squared: 0.808207733542835
MSE:  17514.428530687834
RMSE:  132.34208903704004
MAE: 92.33475673381965
Epoch 1/200
393/393 [==============================] - 3s 4ms/step - loss: 287117.8438
Epoch 2/200
393/393 [==============================] - 2s 5ms/step - loss: 254169.4375
Epoch 3/200
393/393 [==============================] - 2s 4ms/step - loss: 236395.4219
Epoch 4/200
393/393 [==============================] - 2s 4ms/step - loss: 221908.3125
Epoch 5/200
393/393 [==============================] - 2s 4ms/step - loss: 209281.0781
Epoch 6/200
393/393 [==============================] - 1s 4ms/step - loss: 197941.2500
Epoch 7/200
393/393 [==============================] - 2s 4ms/step - loss: 187565.7500
Epoch 8/200
393/393 [==============================] - 2s 4ms/step - loss: 178003.0781
Epoch 9/200
393/393 [==============================] - 2s 4ms/step - loss: 169128.5000
Epoch 10/200
393/393 [==============================] - 1s 4ms/step - loss: 160896.7188
Epoch 11/200
393/393 [====

393/393 [==============================] - 2s 4ms/step - loss: 18145.6934
Epoch 94/200
393/393 [==============================] - 2s 4ms/step - loss: 17946.3789
Epoch 95/200
393/393 [==============================] - 2s 5ms/step - loss: 17991.8984
Epoch 96/200
393/393 [==============================] - 2s 5ms/step - loss: 17918.2910
Epoch 97/200
393/393 [==============================] - 2s 5ms/step - loss: 17525.0176
Epoch 98/200
393/393 [==============================] - 2s 5ms/step - loss: 17526.8809
Epoch 99/200
393/393 [==============================] - 1s 4ms/step - loss: 17473.9492
Epoch 100/200
393/393 [==============================] - 2s 5ms/step - loss: 17296.5918
Epoch 101/200
393/393 [==============================] - 2s 4ms/step - loss: 17290.5840
Epoch 102/200
393/393 [==============================] - 2s 4ms/step - loss: 17183.8965
Epoch 103/200
393/393 [==============================] - 1s 3ms/step - loss: 16943.2559
Epoch 104/200
393/393 [=============================

393/393 [==============================] - 2s 4ms/step - loss: 13917.0020
Epoch 187/200
393/393 [==============================] - 1s 4ms/step - loss: 13943.3730
Epoch 188/200
393/393 [==============================] - 1s 4ms/step - loss: 13981.3164
Epoch 189/200
393/393 [==============================] - 2s 4ms/step - loss: 13929.8174
Epoch 190/200
393/393 [==============================] - 2s 4ms/step - loss: 13763.7793
Epoch 191/200
393/393 [==============================] - 2s 4ms/step - loss: 13746.9463
Epoch 192/200
393/393 [==============================] - 2s 6ms/step - loss: 13898.1406
Epoch 193/200
393/393 [==============================] - 2s 4ms/step - loss: 13717.2334
Epoch 194/200
393/393 [==============================] - 2s 4ms/step - loss: 13789.2910
Epoch 195/200
393/393 [==============================] - 2s 4ms/step - loss: 14401.9375
Epoch 196/200
393/393 [==============================] - 2s 5ms/step - loss: 13678.2305
Epoch 197/200
393/393 [=======================

393/393 [==============================] - 1s 2ms/step - loss: 39317.0195
Epoch 78/200
393/393 [==============================] - 1s 2ms/step - loss: 38554.7617
Epoch 79/200
393/393 [==============================] - 1s 2ms/step - loss: 37970.2891
Epoch 80/200
393/393 [==============================] - 1s 2ms/step - loss: 37415.6562
Epoch 81/200
393/393 [==============================] - 1s 2ms/step - loss: 37091.5117
Epoch 82/200
393/393 [==============================] - 1s 2ms/step - loss: 36645.3281
Epoch 83/200
393/393 [==============================] - 1s 2ms/step - loss: 36299.0898
Epoch 84/200
393/393 [==============================] - 1s 2ms/step - loss: 35887.2734
Epoch 85/200
393/393 [==============================] - 1s 1ms/step - loss: 35599.4102
Epoch 86/200
393/393 [==============================] - 1s 2ms/step - loss: 35288.7422
Epoch 87/200
393/393 [==============================] - 1s 2ms/step - loss: 34956.7148
Epoch 88/200
393/393 [==============================] - 

393/393 [==============================] - 1s 2ms/step - loss: 22273.0527
Epoch 171/200
393/393 [==============================] - 1s 2ms/step - loss: 22221.0215
Epoch 172/200
393/393 [==============================] - 1s 3ms/step - loss: 22147.6523
Epoch 173/200
393/393 [==============================] - 1s 2ms/step - loss: 21953.5586
Epoch 174/200
393/393 [==============================] - 1s 2ms/step - loss: 22001.6816
Epoch 175/200
393/393 [==============================] - 1s 2ms/step - loss: 21885.7715
Epoch 176/200
393/393 [==============================] - 1s 2ms/step - loss: 21778.5039
Epoch 177/200
393/393 [==============================] - 1s 2ms/step - loss: 21732.7949
Epoch 178/200
393/393 [==============================] - 1s 1ms/step - loss: 21598.1895
Epoch 179/200
393/393 [==============================] - 1s 2ms/step - loss: 21481.7051
Epoch 180/200
393/393 [==============================] - 1s 2ms/step - loss: 21403.8984
Epoch 181/200
393/393 [=======================

In [69]:
model_outputs_nn2 = pd.DataFrame(model_outputs_nn2)
model_outputs_nn2

,MLP,LSTM,RNN
R-squared,0.808208,0.829607,0.772518
MSE,17514.428531,15560.235650,20773.617585
RMSE,132.342089,124.740674,144.130557
MAE,92.334757,86.815327,95.989867


In [71]:
sample_predicted_nn2

{'MLP': array([ 226.41790393,  629.8616885 ,  693.95025421, 1256.72970217,
         407.72008236,  535.9172803 ,  619.76438388,  695.20241333,
         576.22353849,  564.64696536,  242.07330215]),
 'LSTM': array([[39.393127],
        [31.703161],
        [33.75795 ],
        [26.254454],
        [34.082535],
        [37.93837 ],
        [35.941097],
        [35.847782],
        [32.09632 ],
        [31.499521],
        [34.789482]], dtype=float32),
 'RNN': array([[283.1871],
        [283.1871],
        [283.1871],
        [283.1871],
        [283.1871],
        [283.1871],
        [283.1871],
        [283.1871],
        [283.1871],
        [283.1871],
        [283.1871]], dtype=float32)}

In [72]:
sample_predicted_nn2['LSTM'] = sample_predicted_nn2['LSTM'].flatten()
sample_predicted_nn2['RNN'] = sample_predicted_nn2['RNN'].flatten()

In [73]:
sample_predicted_nn2 = pd.DataFrame(sample_predicted_nn2)
real_pred_nn2=pd.concat([a[['prıce']],sample_predicted_nn2], axis=1)
real_pred_nn2

,prıce,MLP,LSTM,RNN
0,116.47,226.417904,39.393127,283.187103
1,203.02,629.861688,31.703161,283.187103
2,1038.15,693.950254,33.757950,283.187103
3,1307.82,1256.729702,26.254454,283.187103
4,674.56,407.720082,34.082535,283.187103
5,243.50,535.917280,37.938370,283.187103
6,303.87,619.764384,35.941097,283.187103
7,184.08,695.202413,35.847782,283.187103
8,275.78,576.223538,32.096321,283.187103
9,327.88,564.646965,31.499521,283.187103
